In [1]:
import pandas as pd
import numpy as np

In [2]:
customers = pd.read_csv('./data/customers.csv', parse_dates = ['created'])
customers.head()

,id,created
0,35410,2015-07-03 22:01:11
1,35417,2015-07-03 22:11:23
2,35412,2015-07-03 22:02:52
3,35413,2015-07-03 22:05:02
4,35424,2015-07-03 22:21:55


In [3]:
customers.shape

(25716, 2)

In [4]:
orders = pd.read_csv('./data/orders.csv', parse_dates = ['created'])
orders.head()

,id,order_number,user_id,created
0,1709,36,344,2014-10-28 00:20:01
1,1406,7,608,2014-10-14 23:44:53
2,1716,6,2296,2014-10-28 17:47:07
3,1426,2,1225,2014-10-15 18:33:38
4,1415,6,797,2014-10-15 02:07:16


In [5]:
orders.shape

(27575, 4)

In [6]:
customers_orders = customers.merge(orders, left_on='id', right_on='user_id', suffixes=('_customer', '_order'))
customers_orders.head()

,id_customer,created_customer,id_order,order_number,user_id,created_order
0,35424,2015-07-03 22:21:55,27970,1,35424,2015-07-03 23:37:49
1,35399,2015-07-03 21:30:36,27940,1,35399,2015-07-03 22:17:24
2,35414,2015-07-03 22:09:04,27949,1,35414,2015-07-03 22:51:05
3,35452,2015-07-04 00:20:15,27988,1,35452,2015-07-04 00:30:13
4,35442,2015-07-03 23:33:17,27980,1,35442,2015-07-04 00:05:48


In [7]:
customers_orders.shape

(17015, 6)

In [8]:
customers_orders = customers_orders.rename(columns = {'created_customer': 'customer_cohort'})
customers_orders.head()

,id_customer,customer_cohort,id_order,order_number,user_id,created_order
0,35424,2015-07-03 22:21:55,27970,1,35424,2015-07-03 23:37:49
1,35399,2015-07-03 21:30:36,27940,1,35399,2015-07-03 22:17:24
2,35414,2015-07-03 22:09:04,27949,1,35414,2015-07-03 22:51:05
3,35452,2015-07-04 00:20:15,27988,1,35452,2015-07-04 00:30:13
4,35442,2015-07-03 23:33:17,27980,1,35442,2015-07-04 00:05:48


In [9]:
customers_orders.shape

(17015, 6)

### get the number of distinct customers that ordered in each cohort in each week

In [10]:
# groupby created column to get distinct # of customers that ordered in a week?
n_distinct_user_orders_by_week = customers_orders.groupby([pd.Grouper(key='customer_cohort', freq='W'), pd.Grouper(key='created_order', freq='W')]).agg({'user_id': pd.Series.nunique}).rename(columns={'user_id': 'n_distinct_customers'})
n_distinct_user_orders_by_week.head()

n_distinct_customers
customer_cohort created_order                      
2015-01-04      2015-01-04                       16
                2015-01-11                        8
                2015-01-18                       11
                2015-01-25                        9
                2015-02-01                        9

### Get the count of distinct customers that ordered for the first time by week

In [11]:
# first group by the customer_id
first_order_by_customer = orders.groupby('user_id').agg({'created': np.min}).rename(columns = {'created': 'first_order'}) # time of first order by user id
first_order_by_customer = first_order_by_customer.reset_index()

# then, group by the first order week, count the number of user ids that fall within that week
count_first_order_by_week = first_order_by_customer.groupby(pd.Grouper(key = 'first_order', freq = 'W'))\
                                                   .agg({'user_id': pd.Series.nunique})\
                                                   .rename(columns={'user_id': 'first_order_count'})
count_first_order_by_week.head()

,first_order_count
first_order,
2014-05-18,2
2014-05-25,1
2014-06-01,1
2014-06-08,1
2014-06-15,0


### Merge the first order by customer id table (id is the index) with the customers_orders join table so that we can match when the customer made their first order

In [12]:
customers_orders_w_first = customers_orders.merge(first_order_by_customer, left_on='id_customer', right_on='user_id', how='inner')
customers_orders_w_first.head()

,id_customer,customer_cohort,id_order,order_number,user_id_x,created_order,user_id_y,first_order
0,35424,2015-07-03 22:21:55,27970,1,35424,2015-07-03 23:37:49,35424,2015-07-03 23:37:49
1,35399,2015-07-03 21:30:36,27940,1,35399,2015-07-03 22:17:24,35399,2015-07-03 22:17:24
2,35414,2015-07-03 22:09:04,27949,1,35414,2015-07-03 22:51:05,35414,2015-07-03 22:51:05
3,35452,2015-07-04 00:20:15,27988,1,35452,2015-07-04 00:30:13,35452,2015-07-04 00:30:13
4,35442,2015-07-03 23:33:17,27980,1,35442,2015-07-04 00:05:48,35442,2015-07-04 00:05:48


In [13]:
customers_orders.shape

(17015, 6)

Make a boolean column that marks whether an order is the first one?
when we group by the cohort and created order dates, count the number of trues

In [14]:
customers_orders_w_first['is_first'] = customers_orders_w_first.apply(lambda x: 1 if x['created_order'] == x['first_order'] else 0, axis = 1)
customers_orders_w_first.head()

,id_customer,customer_cohort,id_order,order_number,user_id_x,created_order,user_id_y,first_order,is_first
0,35424,2015-07-03 22:21:55,27970,1,35424,2015-07-03 23:37:49,35424,2015-07-03 23:37:49,1
1,35399,2015-07-03 21:30:36,27940,1,35399,2015-07-03 22:17:24,35399,2015-07-03 22:17:24,1
2,35414,2015-07-03 22:09:04,27949,1,35414,2015-07-03 22:51:05,35414,2015-07-03 22:51:05,1
3,35452,2015-07-04 00:20:15,27988,1,35452,2015-07-04 00:30:13,35452,2015-07-04 00:30:13,1
4,35442,2015-07-03 23:33:17,27980,1,35442,2015-07-04 00:05:48,35442,2015-07-04 00:05:48,1


In [15]:
customers_orders_w_first['is_first'].value_counts()

0    11659
1     5356
Name: is_first, dtype: int64

## get the sum of customer who made their first order, and the count of the unique customers who ordered in that week


In [16]:
cohorts_w_all_counts = customers_orders_w_first\
.groupby([pd.Grouper(key='customer_cohort', freq='W'), pd.Grouper(key='created_order', freq='W')])\
.agg({'id_customer': pd.Series.nunique, 'is_first': np.sum})\
.rename(columns = {'id_customer': 'unique_customers', 'is_first': 'n_first_orders'})

cohorts_w_all_counts.head()

unique_customers  n_first_orders
customer_cohort created_order                                  
2015-01-04      2015-01-04                   16              16
                2015-01-11                    8               4
                2015-01-18                   11               3
                2015-01-25                    9               1
                2015-02-01                    9               1

### Get the number of customers in each cohort

In [17]:
n_customers_by_cohort = customers_orders_w_first.groupby(pd.Grouper(key = 'customer_cohort', freq = 'W'))\
.agg({'id_customer': pd.Series.nunique})\
.rename(columns = {'id_customer': 'n_customers'})
n_customers_by_cohort.head()

,n_customers
customer_cohort,
2015-01-04,32
2015-01-11,389
2015-01-18,394
2015-01-25,469
2015-02-01,234


### the number of customers in each cohort should be less than or equal to the sum of the number of first orders

In [18]:
print('sum of first orders for cohort of 2015-01-04: ', 
      cohorts_w_all_counts.loc['2015-06-07']['n_first_orders'].sum(),
      '\nnumber of customer in cohort 2015-01-04: ',
      n_customers_by_cohort.loc['2015-06-07']['n_customers'])

sum of first orders for cohort of 2015-01-04:  199 
number of customer in cohort 2015-01-04:  199


### Merge the n_customers_by_cohort and the cohorts_w_all_counts dataframes to get the total customers in to the dataframe

In [19]:
cohorts_w_all_counts = cohorts_w_all_counts.merge(n_customers_by_cohort, right_index = True, left_index = True)
cohorts_w_all_counts.head()

unique_customers  n_first_orders  n_customers
customer_cohort created_order                                               
2015-01-04      2015-01-04                   16              16           32
                2015-01-11                    8               4           32
                2015-01-18                   11               3           32
                2015-01-25                    9               1           32
                2015-02-01                    9               1           32

In [20]:
cohorts_w_all_counts = cohorts_w_all_counts.reset_index().groupby([pd.Grouper(key='customer_cohort', freq='W'), 
                                             pd.Grouper(key='created_order', freq='W'), 
                                             'n_customers'])\
                                            .agg({ 'n_first_orders': np.sum, 
                                                  'unique_customers': np.sum,
                                                  'n_customers': np.sum
                                                 })
# test.head()

### Get the percentage of distinct customers ordered within X days from their signup date, where X is a multiple of 7 and the percentage of those customers who were ordering for the first time

In [21]:
cohorts_w_all_counts['percent_orderers'] = cohorts_w_all_counts['unique_customers'] / cohorts_w_all_counts['n_customers']
cohorts_w_all_counts['percent_first_time'] = cohorts_w_all_counts['n_first_orders'] / cohorts_w_all_counts['n_customers']
cohorts_w_all_counts.head()

n_first_orders  unique_customers  \
customer_cohort created_order n_customers                                     
2015-01-04      2015-01-04    32                       16                16   
                2015-01-11    32                        4                 8   
                2015-01-18    32                        3                11   
                2015-01-25    32                        1                 9   
                2015-02-01    32                        1                 9   

                                           n_customers  percent_orderers  \
customer_cohort created_order n_customers                                  
2015-01-04      2015-01-04    32                    32           0.50000   
                2015-01-11    32                    32           0.25000   
                2015-01-18    32                    32           0.34375   
                2015-01-25    32                    32           0.28125   
                2015-02-01    32                    32           0.28125   

                                           percent_first_time  
customer_cohort created_order n_customers                      
2015-01-04      2015-01-04    32                      0.50000  
                2015-01-11    32                      0.12500  
                2015-01-18    32                      0.09375  
                2015-01-25    32                      0.03125  
                2015-02-01    32                      0.03125

### Define formatting functions for orderers and for first timers

In [22]:
def format_orderers(data):
    return "{:.0%} orderers ({:.0f})".format(data['percent_orderers'], data['unique_customers'])

def format_firsts(data):
    return "{:.0%} 1st time ({:.0f})".format(data['percent_first_time'], data['n_first_orders'])

In [23]:
cohorts_w_all_counts['formatted_percent_orderers'] = cohorts_w_all_counts.apply(format_orderers, axis = 1)
cohorts_w_all_counts['formatted_percent_first_time'] = cohorts_w_all_counts.apply(format_firsts, axis = 1)
cohorts_w_all_counts.head()

n_first_orders  unique_customers  \
customer_cohort created_order n_customers                                     
2015-01-04      2015-01-04    32                       16                16   
                2015-01-11    32                        4                 8   
                2015-01-18    32                        3                11   
                2015-01-25    32                        1                 9   
                2015-02-01    32                        1                 9   

                                           n_customers  percent_orderers  \
customer_cohort created_order n_customers                                  
2015-01-04      2015-01-04    32                    32           0.50000   
                2015-01-11    32                    32           0.25000   
                2015-01-18    32                    32           0.34375   
                2015-01-25    32                    32           0.28125   
                2015-02-01    32                    32           0.28125   

                                           percent_first_time  \
customer_cohort created_order n_customers                       
2015-01-04      2015-01-04    32                      0.50000   
                2015-01-11    32                      0.12500   
                2015-01-18    32                      0.09375   
                2015-01-25    32                      0.03125   
                2015-02-01    32                      0.03125   

                                          formatted_percent_orderers  \
customer_cohort created_order n_customers                              
2015-01-04      2015-01-04    32                   50% orderers (16)   
                2015-01-11    32                    25% orderers (8)   
                2015-01-18    32                   34% orderers (11)   
                2015-01-25    32                    28% orderers (9)   
                2015-02-01    32                    28% orderers (9)   

                                          formatted_percent_first_time  
customer_cohort created_order n_customers                               
2015-01-04      2015-01-04    32                     50% 1st time (16)  
                2015-01-11    32                      12% 1st time (4)  
                2015-01-18    32                       9% 1st time (3)  
                2015-01-25    32                       3% 1st time (1)  
                2015-02-01    32                       3% 1st time (1)

### Pivot out the orderers, the first times, and the cohort customers

In [24]:
cohorts_w_all_counts.index = cohorts_w_all_counts.index.set_names('cohort_total', level = 2)
cohorts_all_pivot = cohorts_w_all_counts.reset_index()

In [25]:
orderers_pivot = cohorts_all_pivot.pivot(index = 'customer_cohort', columns = 'created_order', values = 'formatted_percent_orderers')
# reorder the columns so that the later dates come first
orderers_pivot = orderers_pivot.reindex(sorted(orderers_pivot.columns, reverse = True), axis = 1).sort_index(ascending = False)

first_time_pivot = cohorts_all_pivot.pivot(index = 'customer_cohort', columns = 'created_order', values = 'formatted_percent_first_time')
first_time_pivot = first_time_pivot.reindex(sorted(first_time_pivot.columns, reverse = True), axis = 1).sort_index(ascending = False)

n_customers_pivot = cohorts_all_pivot.pivot(index = 'customer_cohort', columns = 'created_order', values = 'n_customers')\
                    .reindex(sorted(first_time_pivot.columns, reverse = True), axis = 1)\
                    .sort_index(ascending = False)\
                    .fillna('')


In [26]:
n_customers_pivot['Customers'] = n_customers_pivot[pd.to_datetime('2015-07-12')].astype(int)

In [27]:
n_customers_pivot

created_order,2015-07-12 00:00:00,2015-07-05 00:00:00,2015-06-28 00:00:00,2015-06-21 00:00:00,2015-06-14 00:00:00,2015-06-07 00:00:00,2015-05-31 00:00:00,2015-05-24 00:00:00,2015-05-17 00:00:00,2015-05-10 00:00:00,...,2015-03-01 00:00:00,2015-02-22 00:00:00,2015-02-15 00:00:00,2015-02-08 00:00:00,2015-02-01 00:00:00,2015-01-25 00:00:00,2015-01-18 00:00:00,2015-01-11 00:00:00,2015-01-04 00:00:00,Customers
customer_cohort,,,,,,,,,,,,,,,,,,,,,
2015-07-12,37.0,,,,,,,,,,...,,,,,,,,,,37
2015-07-05,155.0,155,,,,,,,,,...,,,,,,,,,,155
2015-06-28,164.0,164,164,,,,,,,,...,,,,,,,,,,164
2015-06-21,197.0,197,197,197,,,,,,,...,,,,,,,,,,197
2015-06-14,175.0,175,175,175,175,,,,,,...,,,,,,,,,,175
2015-06-07,199.0,199,199,199,199,199,,,,,...,,,,,,,,,,199
2015-05-31,170.0,170,170,170,170,170,170,,,,...,,,,,,,,,,170
2015-05-24,179.0,179,179,179,179,179,179,179,,,...,,,,,,,,,,179
2015-05-17,158.0,158,158,158,158,158,158,158,158,,...,,,,,,,,,,158


### concatenate the orderers and the first times, sort by the index

In [28]:
first_pass = pd.concat([orderers_pivot, first_time_pivot]).sort_index(ascending=False).fillna('')
first_pass.head()

created_order,2015-07-12,2015-07-05,2015-06-28,2015-06-21,2015-06-14,2015-06-07,2015-05-31,2015-05-24,2015-05-17,2015-05-10,...,2015-03-08,2015-03-01,2015-02-22,2015-02-15,2015-02-08,2015-02-01,2015-01-25,2015-01-18,2015-01-11,2015-01-04
customer_cohort,,,,,,,,,,,,,,,,,,,,,
2015-07-12,100% orderers (37),,,,,,,,,,...,,,,,,,,,,
2015-07-12,100% 1st time (37),,,,,,,,,,...,,,,,,,,,,
2015-07-05,19% 1st time (30),81% 1st time (125),,,,,,,,,...,,,,,,,,,,
2015-07-05,31% orderers (48),81% orderers (125),,,,,,,,,...,,,,,,,,,,
2015-06-28,7% 1st time (11),10% 1st time (17),83% 1st time (136),,,,,,,,...,,,,,,,,,,


### concatenate the number of cohort customers pivot table and the orderers and first times table

In [29]:
final_df = pd.concat([n_customers_pivot, first_pass]).sort_index(ascending = False)
final_df[:20]

created_order,2015-01-04 00:00:00,2015-01-11 00:00:00,2015-01-18 00:00:00,2015-01-25 00:00:00,2015-02-01 00:00:00,2015-02-08 00:00:00,2015-02-15 00:00:00,2015-02-22 00:00:00,2015-03-01 00:00:00,2015-03-08 00:00:00,...,2015-05-17 00:00:00,2015-05-24 00:00:00,2015-05-31 00:00:00,2015-06-07 00:00:00,2015-06-14 00:00:00,2015-06-21 00:00:00,2015-06-28 00:00:00,2015-07-05 00:00:00,2015-07-12 00:00:00,Customers
customer_cohort,,,,,,,,,,,,,,,,,,,,,
2015-07-12,,,,,,,,,,,...,,,,,,,,,37,37.0
2015-07-12,,,,,,,,,,,...,,,,,,,,,100% orderers (37),NaN
2015-07-12,,,,,,,,,,,...,,,,,,,,,100% 1st time (37),NaN
2015-07-05,,,,,,,,,,,...,,,,,,,,81% 1st time (125),19% 1st time (30),NaN
2015-07-05,,,,,,,,,,,...,,,,,,,,81% orderers (125),31% orderers (48),NaN
2015-07-05,,,,,,,,,,,...,,,,,,,,155,155,155.0
2015-06-28,,,,,,,,,,,...,,,,,,,83% 1st time (136),10% 1st time (17),7% 1st time (11),NaN
2015-06-28,,,,,,,,,,,...,,,,,,,83% orderers (136),22% orderers (36),15% orderers (24),NaN
2015-06-28,,,,,,,,,,,...,,,,,,,164,164,164,164.0


In [33]:
final_df[-1:]

created_order,2015-01-04 00:00:00,2015-01-11 00:00:00,2015-01-18 00:00:00,2015-01-25 00:00:00,2015-02-01 00:00:00,2015-02-08 00:00:00,2015-02-15 00:00:00,2015-02-22 00:00:00,2015-03-01 00:00:00,2015-03-08 00:00:00,...,2015-05-17 00:00:00,2015-05-24 00:00:00,2015-05-31 00:00:00,2015-06-07 00:00:00,2015-06-14 00:00:00,2015-06-21 00:00:00,2015-06-28 00:00:00,2015-07-05 00:00:00,2015-07-12 00:00:00,Customers
customer_cohort,,,,,,,,,,,,,,,,,,,,,
2015-01-04,50% 1st time (16),12% 1st time (4),9% 1st time (3),3% 1st time (1),3% 1st time (1),3% 1st time (1),0% 1st time (0),3% 1st time (1),0% 1st time (0),3% 1st time (1),...,0% 1st time (0),0% 1st time (0),0% 1st time (0),0% 1st time (0),0% 1st time (0),0% 1st time (0),3% 1st time (1),0% 1st time (0),0% 1st time (0),NaN


In [28]:
final_df.to_csv('cohort_report.csv')

In [29]:
thing = pd.read_csv('cohort_report.csv', keep_default_na=False)
thing.head()

,customer_cohort,2015-07-12,2015-07-05,2015-06-28,2015-06-21,2015-06-14,2015-06-07,2015-05-31,2015-05-24,2015-05-17,...,2015-03-08,2015-03-01,2015-02-22,2015-02-15,2015-02-08,2015-02-01,2015-01-25,2015-01-18,2015-01-11,2015-01-04
0,2015-07-12,37.0,,,,,,,,,...,,,,,,,,,,
1,2015-07-12,100% orderers (37),,,,,,,,,...,,,,,,,,,,
2,2015-07-12,100% 1st time (37),,,,,,,,,...,,,,,,,,,,
3,2015-07-05,19% 1st time (30),81% 1st time (125),,,,,,,,...,,,,,,,,,,
4,2015-07-05,31% orderers (48),81% orderers (125),,,,,,,,...,,,,,,,,,,
